In [1]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key: ")

In [2]:
os.environ["PINECONE_API_KEY"] = getpass.getpass("Pinecone API Key:")

In [3]:
from uuid import uuid4
unique_id = uuid4().hex[0:8]

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"Mecainic - {unique_id}"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("Enter your LangSmith API key:")

In [11]:
from langchain.document_loaders import PyPDFLoader
from dotenv import load_dotenv
import os

load_dotenv()

# car_manual = PyMuPDFLoader('docs/owner_manual.pdf')
car_manual = PyPDFLoader(os.environ.get('pdfurl'))

In [12]:
car_manual_data = car_manual.load()
print(car_manual_data[5])

page_content='4 \nResetting user data for change of\nownership129\nResetting settings in the centre\ndisplay129\nTable showing centre display settings 130\nDriver profiles 131\nSelecting driver profile 132\nRenaming a driver profile 132\nProtect driver profile 133\nLinking remote control key to\ndriver profile133\nResetting settings in the driver\nprofiles134\nMessage in centre display 135\nManaging messages in the centre\ndisplay135\nHandling a message saved from\nthe centre display136\nHead-up display * 137\nActivating and deactivating the\nhead-up display *138\nSettings for head-up display * 139\nVoice recognition 140\nUsing voice recognition 141\nControlling a telephone with voice\nrecognition143\nVoice control of radio and media 144 \nSettings for voice recognition 145LIGHTING\nLighting control 148\nAdjusting light functions via the\ncentre display150\nAdapting the headlamp pattern\nfrom the headlamps150\nAdjusting headlamp level 150\nPosition lamps 151\nDaytime running lights 152

In [13]:
partial_car_manual_data = car_manual_data

print(partial_car_manual_data[0])

page_content="XC60\nOWNER'S MANUAL" metadata={'source': 'https://az685612.vo.msecnd.net/pdfs/20w17/XC60_OwnersManual_MY21_en-GB_TP32005/XC60_OwnersManual_MY21_en-GB_TP32005.pdf', 'page': 0}


In [14]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
import tiktoken

def tiktoken_len(text):
    tokens = tiktoken.encoding_for_model("gpt-3.5-turbo").encode(
        text,
    )
    return len(tokens)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 400,
    chunk_overlap = 50,
    length_function = tiktoken_len,
)

car_manual_chunks = text_splitter.split_documents(partial_car_manual_data)
car_manual_chunks[:5]

[Document(page_content="XC60\nOWNER'S MANUAL", metadata={'source': 'https://az685612.vo.msecnd.net/pdfs/20w17/XC60_OwnersManual_MY21_en-GB_TP32005/XC60_OwnersManual_MY21_en-GB_TP32005.pdf', 'page': 0}),
 Document(page_content="VÄLKOMMEN!\nWe hope you will enjoy many years of driving pleasure in your Volvo.\nThe car has been designed for the safety and comfort of you and\nyour passengers. Volvo strives to design one of the safest cars in the\nworld. Your Volvo is also designed to meet applicable safety and\nenvironmental requirements.\nTo increase your enjoyment of your Volvo, we recommend that you\nread the instructions and maintenance information in this owner'smanual. The owner's manual is also available as a mobile app (Volvo\nManual) and on the Volvo Cars support site (support.volvocars.com).\nWe encourage everyone to always wear their seatbelt in this and\nother cars. Please do not drive if you are under the influence of alco-\nhol or medication – or have an impaired ability to dr

In [15]:
max_chunk_length = 0

for chunk in car_manual_chunks:
    max_chunk_length = max(max_chunk_length, tiktoken_len(chunk.page_content))

max_chunk_length

400

In [16]:
from pinecone import Pinecone, PodSpec
pinecone_client = Pinecone()

pinecone_client.create_index(
    name=os.environ.get('index'),
    dimension=1536,
    metric="cosine",
    spec=PodSpec(
        environment="gcp-starter"
    )
)

while not pinecone_client.describe_index(os.environ.get('index')).status['ready']:
    print("loading")

print(f"index created: {os.environ.get('index')}")

index created: mechainic


In [17]:
from langchain.vectorstores import Pinecone
from langchain_openai import OpenAIEmbeddings

embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")

vector_store = Pinecone.from_documents(car_manual_chunks, embedding_model, index_name=os.environ.get('index'))

In [18]:
retriever = vector_store.as_retriever()

In [19]:
from langchain_core.prompts import ChatPromptTemplate

RAG_PROMPT = """

CONTEXT:
{context}

QUERY:
{question}

You are a car specialist and can only provide your answers from the context. 

Don't tell in your response that you are getting it from the context.

"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

In [20]:
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-3.5-turbo")

In [21]:
from operator import itemgetter
from langchain.schema.runnable import RunnablePassthrough

mecanic_qa_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | model, "context": itemgetter("context")}
)


In [22]:
response = mecanic_qa_chain.invoke({"question" : "What is the purpose of Event Data Recorder?"})

In [23]:
response["response"]

AIMessage(content='The purpose of the Event Data Recorder (EDR) is to register and record data related to traffic accidents or collision-like situations in order to increase understanding of how vehicle systems work in these types of situations.', response_metadata={'token_usage': {'completion_tokens': 41, 'prompt_tokens': 1595, 'total_tokens': 1636}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3bc1b5746c', 'finish_reason': 'stop', 'logprobs': None})

In [24]:
mecanic_qa_chain.invoke({"question" : "what is the primary purpose of Event Data Recorder (EDR)?"})

{'response': AIMessage(content='The primary purpose of Event Data Recorder (EDR) is to register and record data related to traffic accidents or collision-like situations in order to increase understanding of how vehicle systems work in these types of situations.', response_metadata={'token_usage': {'completion_tokens': 41, 'prompt_tokens': 1690, 'total_tokens': 1731}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3bc1b5746c', 'finish_reason': 'stop', 'logprobs': None}),
 'context': [Document(page_content="data related to the following in the event of\ntraffic accidents or collision-like situations:\n\x81How the various systems in the car\nworked\n\x81Whether the driver and passenger seat-\nbelts were fastened/tensioned\n\x81The driver's use of the accelerator or brake\npedal\n\x81The travel speed of the vehicle\nThis information can help us understand the\ncircumstances in which traffic accidents, inju-\nries and damage occur. The EDR only recordsdata when a non-trivial coll

In [36]:
dev_validation_questions = [
    "what is the primary purpose of Event Data Recorder?",
    "how do you manage the centre display?",
    "how do I adjust the door mirrors?"
    "how do I lower the backrests in the rearseat?"
    "how do I lock and unlock the car?"
    "what is a care key?"
]

dev_key_words_it_should_mention = [
    {"must_contain": ["EDR"]},
    {"must_contain": ["centre display", "display"]},
    {"must_contain": ["door mirrors", "door", "mirrors"]},
    {"must_contain": ["backrests", "rearseat"]},
    {"must_contain": ["lock", "unlock"]},
    {"must_contain": ["care", "key"]},
]

In [37]:
from langsmith import Client

langsmith_client = Client()
dataset_name =  f"Mecainic - evaluation dataset {uuid4().hex[0:8]}";

dataset = langsmith_client.create_dataset(
    dataset_name= dataset_name,
    description="Questioning instruction manuals for use on validating mecainic RAG"
)

langsmith_client.create_examples(
    inputs=[{"question": question} for question in dev_validation_questions],
    outputs= dev_key_words_it_should_mention,
    dataset_id=dataset.id
)


In [38]:
from langsmith.evaluation import EvaluationResult, run_evaluator

@run_evaluator
def must_contain(run, example) -> EvaluationResult:
    prediction = run.outputs.get("output") or ""
    required = example.outputs.get("must_contain") or []
    score = all(phrase in prediction for phrase in required)
    return EvaluationResult(key="must_contain", score=score)


In [39]:
from langchain.smith import RunEvalConfig, run_on_dataset

eval_configuration = RunEvalConfig(
    custom_evaluators=[must_contain],
    evaluators=[
        "criteria", #evaluates a model based on custom set of criteria
        "qa", #evaluates answers to questions using an LLM
        "cot_qa", #chain of thought question answering - uses chain of thought
    ],
)

In [40]:
langsmith_client.run_on_dataset(
    dataset_name=dataset_name,
    llm_or_chain_factory=mecanic_qa_chain,
    evaluation=eval_configuration,
    verbose=True,
    project_name=f"Mecainic RAG evaluation - {uuid4().hex[0:8]}",
    project_metadata={"version": "1.0.0"}
)



View the evaluation results for project 'Mecainic RAG evaluation - 3c8c9c9a' at:
https://smith.langchain.com/o/269156a3-d252-5d5c-9e5a-09f79ff84f0a/datasets/244ed1fc-2752-4ff1-a9ab-2f9c59227cc6/compare?selectedSessions=bc1d91da-2a9e-450f-aefc-736e7deb1d8e

View all tests for Dataset Mecainic - evaluation dataset 5062703f at:
https://smith.langchain.com/o/269156a3-d252-5d5c-9e5a-09f79ff84f0a/datasets/244ed1fc-2752-4ff1-a9ab-2f9c59227cc6
[------------------------------------------------->] 3/3

,feedback.helpfulness,feedback.correctness,feedback.COT Contextual Accuracy,feedback.must_contain,error,execution_time,run_id
count,0,0,0,3,0,3.000000,3
unique,0,0,0,1,0,NaN,3
top,NaN,NaN,NaN,False,NaN,NaN,95132308-589a-4278-af21-6f809c6f95c4
freq,NaN,NaN,NaN,3,NaN,NaN,1
mean,NaN,NaN,NaN,NaN,NaN,10.783394,NaN
std,NaN,NaN,NaN,NaN,NaN,3.170581,NaN
min,NaN,NaN,NaN,NaN,NaN,8.778438,NaN
25%,NaN,NaN,NaN,NaN,NaN,8.955722,NaN
50%,NaN,NaN,NaN,NaN,NaN,9.133006,NaN
75%,NaN,NaN,NaN,NaN,NaN,11.785872,NaN


{'project_name': 'Mecainic RAG evaluation - 3c8c9c9a',
 'results': {'eb8c4e09-c9b0-4448-ae62-5d87dd5275d7': {'input': {'question': 'what is the primary purpose of Event Data Recorder?'},
   'feedback': [EvaluationResult(key='helpfulness', score=None, value=None, comment='Error evaluating run 95132308-589a-4278-af21-6f809c6f95c4: Could not map run prediction with multiple keys: {\'response\': AIMessage(content=\'The primary purpose of the Event Data Recorder (EDR) is to register and record data related to traffic accidents or collision-like situations, such as when the airbag deploys or the vehicle strikes an obstacle in the road. The data is recorded to increase understanding of how vehicle systems work in these types of situations.\', response_metadata={\'token_usage\': {\'completion_tokens\': 63, \'prompt_tokens\': 1668, \'total_tokens\': 1731}, \'model_name\': \'gpt-3.5-turbo\', \'system_fingerprint\': \'fp_3bc1b5746c\', \'finish_reason\': \'stop\', \'logprobs\': None}), \'context\'